# Day 2 - Dive!

>Now, you need to figure out how to pilot this thing.
>
>It seems like the submarine can take a series of commands like forward 1, down 2, or up 3:
>
>`forward X` increases the horizontal position by X units.
>`down X` increases the depth by X units.
>`up X` decreases the depth by X units.
>Note that since you're on a submarine, down and up affect your depth, and so they have the opposite result of what you might expect.
>
>The submarine seems to already have a planned course (your puzzle input). You should probably figure out where it's going. For example:
>```
forward 5
down 5
forward 8
up 3
down 8
forward 2
>```
>
>Your horizontal position and depth both start at 0. The steps above would then modify them as follows:
>
>`forward 5` adds 5 to your horizontal position, a total of 5.
>`down 5` adds 5 to your depth, resulting in a value of 5.
>`forward 8` adds 8 to your horizontal position, a total of 13.
>`up 3` decreases your depth by 3, resulting in a value of 2.
>`down 8` adds 8 to your depth, resulting in a value of 10.
>`forward 2` adds 2 to your horizontal position, a total of 15.
>
>After following these instructions, you would have a horizontal position of 15 and a depth of 10. (Multiplying these together produces 150.)

Like every day, we'll start by reading our input data using the helper function in `utils.py`.

In [4]:
from utils import read_input

instructions = read_input(2)

## Part 1

### Named tuples

This challenge is a great place to use named tuples! They make the code much nicer to read and easier to interact with since we have a position of two values (depth and horizontal position) so we could use a list (`[0, 1]`) or a tuple (`(0, 1)`) but we'd always have to try to remember which one is which.

Using [namedtuple](https://docs.python.org/3/library/collections.html#collections.namedtuple) from the `collections` module, we can give our position and its values names and refer to those while being completely compatible with any code that deals with regular tuples.

And not only that, it has a really nice print output so we can just print the tuple itself and not have to wrap it around custom f-strings to showcase what value is what.

In [19]:
from collections import namedtuple

Position = namedtuple('Position', ['depth', 'horizontal'])

### Pattern matching

Finally I get to use [pattern matching](https://hamatti.org/posts/pattern-matching-is-coming-to-python/) that was added to Python in the latest 3.10 release!

Pattern matching is really nice for situations like these where you have a finite set of options.

If you attempt to run the below code yourself and get a syntax error on `match op`, check that you're running 3.10 or later version of Python (you can do it with `python --version` or within Python with

```python
import platform
print(platform.python_version())
```

In [17]:
def calculate_new_position(instructions):
    depth = 0
    horizontal = 0
    
    for instruction in instructions:
        op, delta = instruction.split(' ')
        delta = int(delta)
        match op:
            case 'forward':
                horizontal += delta
            case 'up':
                depth -= delta
            case 'down':
                depth += delta
            case _:
                pass
            
    return Position(depth, horizontal)
            

In [20]:
part1_final_position = calculate_new_position(instructions)
print(f'Final position: {part1_final_position}')

Final position: Position(depth=1030, horizontal=2010)


> Calculate the horizontal position and depth you would have after following the planned course. 
>
> **What do you get if you multiply your final horizontal position by your final depth?**

In [14]:
print(f'Result: {part1_final_position.depth * part1_final_position.horizontal}')

Result: 2070300


## Part 2

>Based on your calculations, the planned course doesn't seem to make any sense. You find the submarine manual and discover that the process is actually slightly more complicated.
>
>In addition to horizontal position and depth, you'll also need to track a third value, aim, which also starts at 0. The commands also mean something entirely different than you first thought:
>
>down X increases your aim by X units.
>up X decreases your aim by X units.
>forward X does two things:
>- It increases your horizontal position by X units.
>- It increases your depth by your aim multiplied by X.
>
>Again note that since you're on a submarine, down and up do the opposite of what you might expect: "down" means aiming in the positive direction.
>
>Now, the above example does something different:
>
>`forward 5` adds 5 to your horizontal position, a total of 5. Because your aim is 0, your depth does not change.
>`down 5` adds 5 to your aim, resulting in a value of 5.
>`forward 8` adds 8 to your horizontal position, a total of 13. Because your aim is 5, your depth increases by 8*5=40.
>`up 3` decreases your aim by 3, resulting in a value of 2.
>`down 8` adds 8 to your aim, resulting in a value of 10.
>`forward 2` adds 2 to your horizontal position, a total of 15. Because your aim is 10, your depth increases by 2*10=20 to a total of 60.
>
>After following these new instructions, you would have a horizontal position of 15 and a depth of 60. (Multiplying these produces 900.)

To adjust to the new process, let's create a new function that takes the aim into account.

In [34]:
def calculate_real_position(instructions):
    depth = 0
    horizontal = 0
    aim = 0
    
    for instruction in instructions:
        op, delta = instruction.split(' ')
        delta = int(delta)
        match op:
            case 'forward':
                horizontal += delta
                depth += aim * delta
            case 'up':
                aim -= delta
            case 'down':
                aim += delta
            case _:
                pass
            
    return Position(depth, horizontal)

In [35]:
part2_final_position = calculate_real_position(instructions)
print(part2_final_position)

Position(depth=1034321, horizontal=2010)


In [36]:
print(f'Result: {part2_final_position.depth * part2_final_position.horizontal}')

Result: 2078985210


### Using a custom transformer function

In the above example, I used `read_input` function merely to read the raw input.

However, it supports using custom transformer functions (I used `int` in day 1) so here's a way we could have front loaded the information parsing to that function to have a bit more cleaner data to work with.

This way, `better_calculate_real_position` function doesn't need to mess with splitting strings and converting to integers but can purely rely on doing what it is supposed to do and calculate the position.

In [50]:
def transformer(instruction):    
    op, delta = instruction.split(' ')
    delta = int(delta)
    
    return [op, delta]

beta_instructions = read_input(2, transformer)

def better_calculate_real_position(instructions):
    depth = 0
    horizontal = 0
    aim = 0
    
    for [op, delta] in instructions:
        match op:
            case 'forward':
                horizontal += delta
                depth += aim * delta
            case 'up':
                aim -= delta
            case 'down':
                aim += delta
            case _:
                pass
            
    return Position(depth, horizontal)

result = better_calculate_real_position(beta_instructions)
print(result)
print(f'Result: {result.depth * result.horizontal}')

Position(depth=1034321, horizontal=2010)
Result: 2078985210
